update_time 으로 끊기

In [13]:
import requests
import json
import pandas as pd
from datetime import datetime


def bunjang(key_word, pages):
    pid = []
    for page in range(pages):
        url = f'https://api.bunjang.co.kr/api/1/find_v2.json?order=date&n=96&page={page}&req_ref=search&q={key_word}&stat_device=w&stat_category_required=1&version=4'
        response = requests.get(url)
        datas = response.json()['list']
        ids = [data['pid'] for data in datas]
        pid.extend(ids)
        items=[]
        for id in pid:
            url = 'https://api.bunjang.co.kr/api/1/product/{}/detail_info.json?version=4'.format(id)
            response = requests.get(url)
            try:
                details = response.json()['item_info']
                details.pop('category_name')
                details.pop('pay_option')
                items.append(details)
            except:
                print('error')
        df = pd.DataFrame(items)
        bunjang_df = df[['name','price','location','description_for_detail','num_item_view','pid']]
        
        bunjang_df['url'] = 'https://m.bunjang.co.kr/products/'+ bunjang_df['pid']
        bunjang_df['market'] = '번개장터'
        bunjang_df['keyword'] = key_word
#         bunjang_df.drop(['pid'], axis=1)

        
        return bunjang_df
    
categories = ["자전거","패딩","노트북","의자","아이폰","아이패드","캠핑","냉장고","컴퓨터","난로","에어팟","모니터"]
a=[]
for category in categories:
    a.append(bunjang(category,5))

<ipython-input-13-e13e95ccce1f>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bunjang_df['url'] = 'https://m.bunjang.co.kr/products/'+ bunjang_df['pid']
<ipython-input-13-e13e95ccce1f>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bunjang_df['market'] = '번개장터'
<ipython-input-13-e13e95ccce1f>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [14]:
a[0]

,name,price,location,description_for_detail,num_item_view,pid,url,market,keyword
0,홍진 발레코 자전거 헬멧 M,165000,경기도 고양시 일산동구 장항2동,브랜드 : 홍진 HJC SPORTS\n모델 : 발레코\n컬러 : 실버 화이트\...,324,180914035,https://m.bunjang.co.kr/products/180914035,번개장터,자전거
1,홍진 HJC 발레코 자전거 헬멧 L,160000,경기도 고양시 일산동구 장항2동,브랜드 : 홍진 HJC SPORTS\n모델 : 발레코\n컬러 : 블랙 옐로우\...,146,180913838,https://m.bunjang.co.kr/products/180913838,번개장터,자전거
2,카스크 정품 프로톤 자전거 헬멧 M / L,260000,경기도 고양시 일산동구 장항2동,모델 : KASK 프로톤 매트 올리브 그린\r\n사이즈 : M(52~58cm) /...,283,171471957,https://m.bunjang.co.kr/products/171471957,번개장터,자전거
3,[가격내림] 크로몰리 프레임 + 자전거 헬멧 무료나눔,35000,경기도 광명시 하안1동,"크로몰리 듣보 프레임셋 + 크랭크셋 입니다.\n녹, 먹음 없이 기스가 좀 있습니다....",1219,169226341,https://m.bunjang.co.kr/products/169226341,번개장터,자전거
4,자전거,50000,서울특별시 성북구 종암동,자전거 상태는 다 좋은데 뒷바퀴 타이어가 수명이 거의 다되서 타이어를 갈아야 할것같...,485,172583282,https://m.bunjang.co.kr/products/172583282,번개장터,자전거
...,...,...,...,...,...,...,...,...,...
91,산마르코 콩코르 슈퍼코르사 안장,55000,경기도 용인시 수지구 동천동,경륜차대에 잘 어울리는 콩코르 슈퍼코르사 안장입니다\n미사용 신품이요,202,182812812,https://m.bunjang.co.kr/products/182812812,번개장터,자전거
92,로드자전거 250만원 안에서 구매합니다,2500000,전라남도 여수시 둔덕동,가격은 이쁘면 조금 더 오버되도 괜찮습니다. 22단에 이쁘면 됩니다.,10,185561572,https://m.bunjang.co.kr/products/185561572,번개장터,자전거
93,트렉 mt 220 자전거,110000,경기도 성남시 분당구 정자1동,Mt220 자전거입니다. 어린이 전용이구요.\n\n10번정도 탔다가 꽤 오랫동안 방...,9,185561305,https://m.bunjang.co.kr/products/185561305,번개장터,자전거
94,자전거 거치대,7000,경기도 하남시 미사2동,자전거 거치대\n신품입니다 \n\n하남미사 직거래 희망합니다,2,185561279,https://m.bunjang.co.kr/products/185561279,번개장터,자전거


In [18]:
page=0
key_word="커피 믹스"
url = f'https://api.bunjang.co.kr/api/1/find_v2.json?order=date&n=96&page={page}&req_ref=search&q={key_word}&stat_device=w&stat_category_required=1&version=4'
data = requests.get(url).json()['list']

In [23]:
df=pd.DataFrame(data)
df

,ad,bizseller,checkout,contact_hope,free_shipping,is_adult,is_super_up_shop,location,max_cpc,name,...,tag,uid,update_time,used,bun_pay_filter_enabled,inspection,imp_id,ad_ref,faved,id
0,False,False,False,False,False,False,None,서울특별시 성동구 성수2가제3동,None,자판기용 맥심모카골드 커피믹스 9개,...,맥심모카골드 자판기커피,78859866,1650369173,2,True,None,27f2625ea967021f908a,,False,0
1,False,False,False,False,True,False,None,대구광역시 남구 봉덕1동,None,미사용)맥심 홈카페 키트 + 믹스커피,...,홈카페 맥심 우드트레이 머그잔 머그컵,3457994,1650367329,2,True,None,27f2625ea967021f908b,,False,1
2,False,False,False,False,True,False,None,충청북도 진천군 진천읍,None,맥심 화이트 커피 믹스 300개+맥심 모카골드 마일드 115개 + 신라면,...,라면 치킨 피자 족발 만두,196711,1650365086,1,True,None,27f2625ea967021f908c,,False,2
3,False,False,False,False,True,False,None,경기도 화성시 남양읍,None,맥심 모카골드 커피믹스 420t gs반값택배 택포,...,,3560384,1650363677,2,True,None,27f2625ea967021f908d,,False,3
4,False,False,False,False,True,False,None,경기도 부천시 중3동,None,맥심 카누 마일드 로스트 아메리카노 미니 원두믹스 30개입 운포 8000,...,커피 아메리카노 원두 라떼 커플,2568754,1650361520,2,True,None,27f2625ea967021f908e,,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,False,False,False,False,False,False,None,,None,애텀 카피믹스,...,커피,9650596,1649765064,1,True,None,27f2625ea967021f90e5,,False,91
92,False,False,False,False,True,False,None,경상북도 포항시 북구 죽도동,None,믹스커피 하이웨스트 슬랙스,...,새상품 무료배송 슬랙스 믹스커피 하이웨스트,75115107,1649758614,2,False,None,27f2625ea967021f90e6,,False,92
93,False,False,False,False,True,False,None,서울특별시 은평구 갈현제1동,None,무료배송) 프렌치카페 400t (200t x 2) 커피믹스 카페믹스,...,커피믹스 커피 프렌치카페,75016450,1649722309,2,True,None,27f2625ea967021f90e7,,False,93
94,False,False,False,False,False,False,None,,None,옛날 빈티지 다방커피잔&믹스커피잔 2개일괄,...,,13102666,1649713376,1,False,None,27f2625ea967021f90e8,,False,94


In [34]:
col =['location', 'name',
       'num_comment', 'num_faved', 'only_neighborhood', 'pid',
       'price', 'product_image', 'status','tag', 'uid', 'update_time', 'used','imp_id']

df[col]

,location,name,num_comment,num_faved,only_neighborhood,pid,price,product_image,status,tag,uid,update_time,used,imp_id
0,서울특별시 성동구 성수2가제3동,자판기용 맥심모카골드 커피믹스 9개,0,1,False,184700359,60000,https://media.bunjang.co.kr/product/184700359_...,0,맥심모카골드 자판기커피,78859866,1650369173,2,27f2625ea967021f908a
1,대구광역시 남구 봉덕1동,미사용)맥심 홈카페 키트 + 믹스커피,0,1,False,156678742,25000,https://media.bunjang.co.kr/product/156678742_...,0,홈카페 맥심 우드트레이 머그잔 머그컵,3457994,1650367329,2,27f2625ea967021f908b
2,충청북도 진천군 진천읍,맥심 화이트 커피 믹스 300개+맥심 모카골드 마일드 115개 + 신라면,0,0,False,185542246,50000,https://media.bunjang.co.kr/product/185542246_...,0,라면 치킨 피자 족발 만두,196711,1650365086,1,27f2625ea967021f908c
3,경기도 화성시 남양읍,맥심 모카골드 커피믹스 420t gs반값택배 택포,0,0,False,185555259,41000,https://media.bunjang.co.kr/product/185555259_...,0,,3560384,1650363677,2,27f2625ea967021f908d
4,경기도 부천시 중3동,맥심 카누 마일드 로스트 아메리카노 미니 원두믹스 30개입 운포 8000,0,13,False,134892384,8000,https://media.bunjang.co.kr/product/134892384_...,0,커피 아메리카노 원두 라떼 커플,2568754,1650361520,2,27f2625ea967021f908e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,,애텀 카피믹스,0,1,False,183597241,23000,https://media.bunjang.co.kr/product/183597241_...,0,커피,9650596,1649765064,1,27f2625ea967021f90e5
92,경상북도 포항시 북구 죽도동,믹스커피 하이웨스트 슬랙스,0,0,False,177454236,14900,https://media.bunjang.co.kr/product/177454236_...,0,새상품 무료배송 슬랙스 믹스커피 하이웨스트,75115107,1649758614,2,27f2625ea967021f90e6
93,서울특별시 은평구 갈현제1동,무료배송) 프렌치카페 400t (200t x 2) 커피믹스 카페믹스,0,2,False,180596092,34000,https://media.bunjang.co.kr/product/180596092_...,0,커피믹스 커피 프렌치카페,75016450,1649722309,2,27f2625ea967021f90e7
94,,옛날 빈티지 다방커피잔&믹스커피잔 2개일괄,0,2,False,167680588,5000,https://media.bunjang.co.kr/product/167680588_...,0,,13102666,1649713376,1,27f2625ea967021f90e8


In [25]:
df.columns

Index(['ad', 'bizseller', 'checkout', 'contact_hope', 'free_shipping',
       'is_adult', 'is_super_up_shop', 'location', 'max_cpc', 'name',
       'num_comment', 'num_faved', 'only_neighborhood', 'outlink_url', 'pid',
       'price', 'product_image', 'pu_id', 'ref_campaign', 'ref_code',
       'ref_medium', 'ref_content', 'ref_source', 'ref_test', 'status',
       'style', 'super_up', 'tag', 'uid', 'update_time', 'used',
       'bun_pay_filter_enabled', 'inspection', 'imp_id', 'ad_ref', 'faved',
       'id'],
      dtype='object')

In [36]:
df['used'].unique()

array([2, 1], dtype=int64)